## 1. <Directed Acyclic Graph 구현을 위한 데이터 전처리 코드>

###   1) 총 시뮬레이션 시간 동안의 노드 간 연기 이동 방향 분석
- Stationary dBN
- Non-stationary dBN

#### (1) 노드 간 연기 이동 전처리 -> 데이터 프레임화

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import IPython

In [2]:
# node(Labelized Variable)별 space aim number(xml space name) -> node numbering dictionary
xml2node_code = {'3252':'r1', '3300':'r2', '3348':'r3', '3396':'r4', '3344':'r5', '3492':'r6',
               '2594':'r7', '2642':'r8', '2695':'r9', '1715':'r10', '1773':'r11', '0390':'r12',
               '0438':'r13', '0491':'r14', '3093':'r15', '3151':'r16', '3204':'r17', '1836':'r18', 
               '1899':'r19', '0909':'r20', '0967':'r21', '1020':'r22', '0080':'r23', '0193':'r24', 
                '0241':'r25', '0294':'r26', '0342':'r27', '2154':'r28', '1275':'r29', '2435':'r30',
               '1556':'r31', '2207':'r32', '2260':'r33', '2313':'r34', '1328':'r35', '1381':'r36',
               '1434':'r37', '2753':'r38', '2806':'r39', '2870':'r40', '2923':'r41', '2976':'r42',
               '3040':'r43', '1947':'r44', '2000':'r45', '1068':'r46', '1121':'r47', '2488':'r48',
               '2541':'r49', '1609':'r50', '1662':'r51', '0549':'r52', '0602':'r53', '0676':'r54',
               '0729':'r55', '0782':'r56', '0856':'r57', '2058':'r58', '3540':'r59', '1179':'r60',
               '3609':'r61', '2366':'r62', '3678':'r63', '1487':'r64', '3774':'r65'}

df_mafldensity_raw = pd.read_csv('d.massfluxdensity/analysis_md_aim1381.csv', encoding='utf-8')
# df_mafldensity_raw: 각 time 당 두 room 사이의 mass flux density 이동량
df_mafldensity_raw['Time'] = df_mafldensity_raw['Time'].map(lambda x: int(str(x).split('.')[0])) # time column 소숫점 자리 제거

# raw dataframe melting : [Time] column data -> id_vars, column에 있는 variables -> var_name, 기존데이터 value값 -> MassFluxDensity(kg/s)
df_mafldensity_raw = df_mafldensity_raw.melt(id_vars=['Time'], var_name = 'roomij(i_j)', value_name = 'MassFluxDensity(kg/s)')

In [3]:
# roomij(i_j)에 있는 값을 '_'로 전부 분리해서 정리
df_mafldensity_raw['room i'] = df_mafldensity_raw['roomij(i_j)'].map(lambda x: str(x).split('_')[1])
df_mafldensity_raw['room j'] = df_mafldensity_raw['roomij(i_j)'].map(lambda x: str(x).split('_')[2])
df_mafldensity_raw = df_mafldensity_raw.drop('roomij(i_j)', axis = 1)
df_mafldensity_raw.replace({'room i':xml2node_code}, inplace = True)
df_mafldensity_raw.replace({'room j':xml2node_code}, inplace = True)

In [4]:
df_mafldensity_raw

,Time,MassFluxDensity(kg/s),room i,room j
0,0,0.000000,r44,r45
1,1,0.000000,r44,r45
2,2,0.000000,r44,r45
3,3,0.000000,r44,r45
4,4,0.000000,r44,r45
...,...,...,...,...
29995,395,0.001794,r65,r57
29996,396,0.001992,r65,r57
29997,397,0.001977,r65,r57
29998,398,0.001714,r65,r57


In [5]:
# 총 시뮬레이션 시간 동안 space 간 연기 이동이 없는 경우가 있는지 확인
df_Smafldensity = df_mafldensity_raw.groupby(by=['room i', 'room j']).sum().reset_index()
if len(df_Smafldensity[df_Smafldensity['MassFluxDensity(kg/s)'] == 0]) == 0:
    pass
else:
    print(len(df_Smafldensity[df_Smafldensity['MassFluxDensity(kg/s)'] == 0]))

In [6]:
df_mafldensity_raw

,Time,MassFluxDensity(kg/s),room i,room j
0,0,0.000000,r44,r45
1,1,0.000000,r44,r45
2,2,0.000000,r44,r45
3,3,0.000000,r44,r45
4,4,0.000000,r44,r45
...,...,...,...,...
29995,395,0.001794,r65,r57
29996,396,0.001992,r65,r57
29997,397,0.001977,r65,r57
29998,398,0.001714,r65,r57


In [7]:
# dataframe 정리
df_mafldensity_raw['room ini'], df_mafldensity_raw['room fin'] = "", ""
df_mafldensity_raw

,Time,MassFluxDensity(kg/s),room i,room j,room ini,room fin
0,0,0.000000,r44,r45,,
1,1,0.000000,r44,r45,,
2,2,0.000000,r44,r45,,
3,3,0.000000,r44,r45,,
4,4,0.000000,r44,r45,,
...,...,...,...,...,...,...
29995,395,0.001794,r65,r57,,
29996,396,0.001992,r65,r57,,
29997,397,0.001977,r65,r57,,
29998,398,0.001714,r65,r57,,


#### (2) 노드 간 연기 이동 "방향" 판단 -> 데이터 프레임 column 추가: 출발 room - [room ini], 도착 room - [room fin]
- relationship modeling.xlsx - room간 방향성을 모델링한 파일
- 예를 들어, r65가 r57보다 위에 있기 때문에, massfluxdensity값이 양수이면 massflux가 일어난 방향이 r57 -> r65임.
- 이것은 DAG 결정에 있어 중요한 증거이기 때문에 분석에 추가하였음.
- 같은 room들 간 이동이어도 매초마다 양수일 수도 음수일 수도 있으므로 더 우세한 쪽의 방향을 따르는 것이 맞음.

###### 이를 위해서 분석 자동화 코드는 다음을 포함한다.
   1) 같은 room들 간 이동에서, massfluxdensity가 양수인 경우의 수와 음수인 경우의 수를 세어 비교
   2) sum값의 부호가 1)의 우세한 부호와 일지하는지 확인

In [8]:
room_relation = pd.read_csv('d.others/relationship_modeling.csv', encoding='utf-8')

index_name = []
for colname in room_relation.columns[:]:
    index_name.append('r{}'.format(colname))
    room_relation.rename(columns = {colname:'r{}'.format(colname)}, inplace=True)

room_relation.index = pd.Series(index_name)

In [9]:
room_relation = room_relation.replace('-', np.nan)
room_relation

,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,...,r56,r57,r58,r59,r60,r61,r62,r63,r64,r65
r1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
r2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
r3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
r4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
r5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
r61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,-1,NaN,NaN,NaN,NaN,NaN
r62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,-1,NaN
r63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,-1
r64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1


In [10]:
# room_relation 데이터 프레임에서 room과의 관계를 읽어, 정리된 df_mafldensity_raw 데이터 프레임에 적용
# 매초(time) 별로 어느 room에서 어떤 room으로 매연입자가 확산하는지를 판단하여 새 컬럼에 기록
df_mafldensity_values_list = df_mafldensity_raw.values.tolist()

for x in df_mafldensity_values_list:
    values_index = df_mafldensity_values_list.index(x)
    
    if int(room_relation.loc[x[2], x[3]]) * x[1] > 0:
        df_mafldensity_raw.loc[values_index, 'room ini'] = x[2]
        df_mafldensity_raw.loc[values_index, 'room fin'] = x[3]
        
    elif int(room_relation.loc[x[2], x[3]]) * x[1] < 0:
        df_mafldensity_raw.loc[values_index, 'room ini'] = x[3]
        df_mafldensity_raw.loc[values_index, 'room fin'] = x[2]
        
    else:
        df_mafldensity_raw.loc[values_index, 'room ini'] = np.nan
        df_mafldensity_raw.loc[values_index, 'room fin'] = np.nan

df_mafldensity_raw

,Time,MassFluxDensity(kg/s),room i,room j,room ini,room fin
0,0,0.000000,r44,r45,NaN,NaN
1,1,0.000000,r44,r45,NaN,NaN
2,2,0.000000,r44,r45,NaN,NaN
3,3,0.000000,r44,r45,NaN,NaN
4,4,0.000000,r44,r45,NaN,NaN
...,...,...,...,...,...,...
29995,395,0.001794,r65,r57,r57,r65
29996,396,0.001992,r65,r57,r57,r65
29997,397,0.001977,r65,r57,r57,r65
29998,398,0.001714,r65,r57,r57,r65


In [11]:
# 전처리된 room 간 연기 이동 데이터프레임 -> out
df_mafldensity_tidy = df_mafldensity_raw.copy()
df_mafldensity_tidy.to_csv("r.preprocessed/1381(36).csv")

df_mafldensity_tidy['room i_j'] = df_mafldensity_tidy['room i']+'_'+df_mafldensity_tidy['room j']
df_mafldensity_tidy.drop(columns=['room i', 'room j'], inplace = True)
df_mafldensity_tidy['room ini_fin'] = df_mafldensity_tidy['room ini']+'_'+df_mafldensity_tidy['room fin']
df_mafldensity_tidy.drop(columns=['room ini', 'room fin'], inplace = True)

In [12]:
df_mafldensity_tidy

,Time,MassFluxDensity(kg/s),room i_j,room ini_fin
0,0,0.000000,r44_r45,NaN
1,1,0.000000,r44_r45,NaN
2,2,0.000000,r44_r45,NaN
3,3,0.000000,r44_r45,NaN
4,4,0.000000,r44_r45,NaN
...,...,...,...,...
29995,395,0.001794,r65_r57,r57_r65
29996,396,0.001992,r65_r57,r57_r65
29997,397,0.001977,r65_r57,r57_r65
29998,398,0.001714,r65_r57,r57_r65


In [13]:
grouped=df_mafldensity_tidy.groupby(['room i_j', 'room ini_fin']).count()
grouped.reset_index(level=['room i_j', 'room ini_fin'], inplace = True)
list_grouped_values = grouped.values.tolist()

In [14]:
list_grouped_values

[['r10_r46', 'r46_r10', 241, 241],
 ['r11_r47', 'r47_r11', 238, 238],
 ['r15_r48', 'r48_r15', 183, 183],
 ['r16_r49', 'r49_r16', 185, 185],
 ['r17_r49', 'r49_r17', 187, 187],
 ['r1_r38', 'r38_r1', 248, 248],
 ['r20_r50', 'r50_r20', 259, 259],
 ['r21_r51', 'r51_r21', 258, 258],
 ['r23_r52', 'r52_r23', 278, 278],
 ['r27_r57', 'r57_r27', 290, 290],
 ['r29_r60', 'r60_r29', 302, 302],
 ['r2_r39', 'r39_r2', 248, 248],
 ['r30_r63', 'r63_r30', 220, 220],
 ['r31_r65', 'r65_r31', 289, 289],
 ['r32_r40', 'r40_r32', 222, 222],
 ['r33_r40', 'r40_r33', 207, 207],
 ['r34_r41', 'r41_r34', 208, 208],
 ['r35_r54', 'r54_r35', 321, 321],
 ['r36_r54', 'r36_r54', 375, 375],
 ['r37_r55', 'r55_r37', 314, 314],
 ['r38_r39', 'r38_r39', 247, 247],
 ['r38_r39', 'r39_r38', 1, 1],
 ['r38_r58', 'r58_r38', 273, 273],
 ['r39_r40', 'r39_r40', 251, 251],
 ['r39_r59', 'r59_r39', 275, 275],
 ['r3_r40', 'r40_r3', 202, 202],
 ['r40_r41', 'r40_r41', 227, 227],
 ['r41_r42', 'r42_r41', 239, 239],
 ['r42_r43', 'r43_r42', 243, 2

In [15]:
smoke_move_relation = []
for li in range(len(list_grouped_values)+1):
    if li+1 < len(list_grouped_values):
        list_this_turn = list_grouped_values[li]
        list_next_turn = list_grouped_values[li+1]
        
        if list_this_turn[0] == list_next_turn[0]:
            if list_this_turn[2] > list_next_turn[2]:
                smoke_move_relation.append(list_this_turn[1])
            elif list_this_turn[2] < list_next_turn[2]:
                smoke_move_relation.append(list_next_turn[1])
            else:
                smoke_move_relation.append('identical')
        
        else:
            reverse_list_this_turn = list_this_turn[1].split('_')[1] + '_' + list_this_turn[1].split('_')[0]
            if smoke_move_relation.count(list_this_turn[1]) == 0 and smoke_move_relation.count(reverse_list_this_turn) == 0:
                
                if len(smoke_move_relation) == 0 or smoke_move_relation[-1] != list_this_turn[0]:
                    smoke_move_relation.append(list_this_turn[1])
                
                    if (list_next_turn == list_grouped_values[-1]) and (smoke_move_relation.count(list_next_turn[1]) == 0):
                        smoke_move_relation.append(list_next_turn[1])
                        
len(smoke_move_relation)

75

In [16]:
pair_of_smr = [tuple(x.split('_')) for x in smoke_move_relation]
pair_of_smr0 = [tuple (y+'_0' for y in tuple(x.split('_'))) for x in smoke_move_relation]
pair_of_smrt = [tuple (y+'_t' for y in tuple(x.split('_'))) for x in smoke_move_relation]

In [17]:
pair_of_smr

[('r46', 'r10'),
 ('r47', 'r11'),
 ('r48', 'r15'),
 ('r49', 'r16'),
 ('r49', 'r17'),
 ('r38', 'r1'),
 ('r50', 'r20'),
 ('r51', 'r21'),
 ('r52', 'r23'),
 ('r57', 'r27'),
 ('r60', 'r29'),
 ('r39', 'r2'),
 ('r63', 'r30'),
 ('r65', 'r31'),
 ('r40', 'r32'),
 ('r40', 'r33'),
 ('r41', 'r34'),
 ('r54', 'r35'),
 ('r36', 'r54'),
 ('r55', 'r37'),
 ('r38', 'r39'),
 ('r58', 'r38'),
 ('r39', 'r40'),
 ('r59', 'r39'),
 ('r40', 'r3'),
 ('r40', 'r41'),
 ('r42', 'r41'),
 ('r43', 'r42'),
 ('r62', 'r42'),
 ('r63', 'r43'),
 ('r44', 'r10'),
 ('r45', 'r44'),
 ('r45', 'r11'),
 ('r58', 'r45'),
 ('r46', 'r12'),
 ('r46', 'r13'),
 ('r47', 'r46'),
 ('r47', 'r14'),
 ('r60', 'r47'),
 ('r48', 'r18'),
 ('r48', 'r49'),
 ('r49', 'r19'),
 ('r41', 'r4'),
 ('r50', 'r18'),
 ('r50', 'r51'),
 ('r51', 'r19'),
 ('r51', 'r22'),
 ('r53', 'r52'),
 ('r52', 'r60'),
 ('r54', 'r53'),
 ('r53', 'r61'),
 ('r54', 'r24'),
 ('r54', 'r55'),
 ('r55', 'r25'),
 ('r55', 'r56'),
 ('r56', 'r26'),
 ('r56', 'r57'),
 ('r58', 'r28'),
 ('r58', 'r59'),
 

#### (3) 노드 간 연기 이동 "방향"으로 시각화 -> DAG 결정

In [ ]:
figures = []
plt.figure(figsize=(20,40))
G = nx.DiGraph()
G.add_edges_from(pair_of_smr)
nx.draw_networkx(G, arrows = True, node_color = 'pink')
figures.append(G)
plt.savefig('r.dag/1381(36).png')

## 2. <PyArgum 라이브러리를 이용한 동적 베이지안 네트워크 구현>

###   1) unStationary dBN (nsDBN)
 - DAG 유지, 확률변화 (CPT 변화 DBN만들기)

In [ ]:
import pyAgrum as gum
import pyAgrum.lib.dynamicBN as gdyn
import pyAgrum.lib.notebook as gnb

In [ ]:
dag_dBN = gum.BayesNet('1381(36)')
dag_nodes = list(G.nodes)
dBN_nodes = []

for node in dag_nodes:
    globals()[node+'_0'] = dag_dBN.add(gum.LabelizedVariable(node+'_0',node+'_0',2))
    globals()[node+'_t'] = dag_dBN.add(gum.LabelizedVariable(node+'_t',node+'_t',2))
    dBN_nodes.append(node+'_0')
    dBN_nodes.append(node+'_t')

In [ ]:
pair_of_self = [(node+'_0', node+'_t') for node in dag_nodes]
for link in pair_of_smr0:
    dag_dBN.addArc(*link)

print(dag_dBN)

for link in pair_of_smrt:
    dag_dBN.addArc(*link)

print(dag_dBN)

for link in pair_of_self:
    dag_dBN.addArc(*link)

In [ ]:
for i in dBN_nodes:
    print(dag_dBN.cpt(i))

In [ ]:
T = 8

gnb.showBN(dag_dBN)

ftts_dbn = gdyn.unroll2TBN(dag_dBN, T)
gdyn.showTimeSlices(ftts_dbn, size="30")

In [ ]:
ftts_dbn_nodes = ftts_dbn.names()
ts_0 = sorted([e for e in ftts_dbn_nodes
          if (int(e.split('_')[1]) == 0)])
ts_1 = sorted([e for e in ftts_dbn_nodes
          if (int(e.split('_')[1]) in range(1, 2))])

In [ ]:
ts_2to3 = sorted([e for e in ftts_dbn_nodes
          if (int(e.split('_')[1]) in range(2, 4))])
ts_4to5 = sorted([e for e in ftts_dbn_nodes
          if (int(e.split('_')[1]) in range(4, 6))])
ts_6to7 = sorted([e for e in ftts_dbn_nodes
          if (int(e.split('_')[1]) in range(6, 8))])

### 2) CPT timeslice별 리스트 작성 (CPT_calculation_template_EXP.xlsx 템플릿 이용)

####  (1) CPT를 엑셀 템플릿으로 계산할 때는 총 400초의 simulation time을 100초 단위로 끊어 네 구간에서 계산
####  (2) pyAgrum을 이용한 dBN을 만들 때에는 400초를 8구간으로 나누어 생각함. (50s로 단위로 끊은 후 두 구간씩 같은 list를 사용한다.)

In [ ]:
cpt_0 = [[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[1, 0],[0.0100,0.9900,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0100,0.9900,0.0045,0.9955,0.0055,0.9945,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.0100,0.9900,0.0044,0.9956,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000]]

cpt_1= [[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[1.0000,0.0000,0.3155,0.6845,0.2213,0.7787,0.0000,1.0000],[1.0000,0.0000,0.4833,0.5167,0.0000,1.0000,0.0000,1.0000],[1.0000,0.0000,0.5650,0.4350,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[1.0000,0.0000,0.6328,0.3672,0.3340,0.6660,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.7612,0.2388,0.4710,0.5290,0.2902,0.7098,0.0000,1.0000],[0.0100,0.9900,0.0000,1.0000],[0.7960,0.2040,0.4863,0.5137,0.3097,0.6903,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[1.0000,0.0000,0.0000,1.0000,0.4370,0.5630,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[1.0000,0.0000,0.3581,0.6419,0.3419,0.6581,0.0000,1.0000],[1.0000,0.0000,0.2208,0.7792,0.2343,0.7657,0.0000,1.0000],[1.0000,0.0000,0.1115,0.8885,0.1338,0.8662,0.0000,1.0000],[1.0000,0.0000,0.1679,0.8321,0.1925,0.8075,0.0000,1.0000],[1.0000,0.0000,0.3050,0.6950,0.3050,0.6950,0.0000,1.0000],[1.0000,0.0000,0.4244,0.5756,0.3956,0.6044,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[1.0000,0.0000,0.5143,0.4857,0.5718,0.4282,0.2805,0.7195,0.5251,0.4749,0.2338,0.7662,0.2913,0.7087,0.0000,1.0000],[1.0000,0.0000,0.3690,0.6310,0.3199,0.6801,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[1.0000,0.0000,0.4445,0.5555,0.3498,0.6502,0.0000,1.0000],[1.0000,0.0000,0.5672,0.4328,0.6401,0.3599,0.3166,0.6834,0.5740,0.4260,0.2506,0.7494,0.3234,0.6766,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000],[0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000]]
cpt_2to3 = [[1.0000,0.0000,0.5321,0.4679,0.5813,0.4187,0.3614,0.6386,0.3907,0.6093,0.1708,0.8292,0.2199,0.7801,0.0000,1.0000],[1.0000,0.0000,0.5799,0.4201,0.4710,0.5290,0.2925,0.7075,0.4659,0.5341,0.2874,0.7126,0.1785,0.8215,0.0000,1.0000],[1.0000,0.0000,0.4841,0.5159,0.2325,0.7675,0.0000,1.0000],[1.0000,0.0000,0.4132,0.5868,0.2568,0.7432,0.0000,1.0000],[1.0000,0.0000,0.3926,0.6074,0.2468,0.7532,0.0000,1.0000],[1.0000,0.0000,0.5964,0.4036,0.4144,0.5856,0.0000,1.0000],[1.0000,0.0000,0.6652,0.3348,0.3841,0.6159,0.0000,1.0000],[1.0000,0.0000,0.7324,0.2676,0.3257,0.6743,0.0000,1.0000],[1.0000,0.0000,0.4808,0.5192,0.6333,0.3667,0.2979,0.7021,0.5184,0.4816,0.1829,0.8171,0.3355,0.6645,0.0000,1.0000],[1.0000,0.0000,0.5540,0.4460,0.5898,0.4102,0.3434,0.6566,0.4569,0.5431,0.2106,0.7894,0.2464,0.7536,0.0000,1.0000],[1.0000,0.0000,0.4915,0.5085,0.2586,0.7414,0.0000,1.0000],[1.0000,0.0000,0.4135,0.5865,0.2599,0.7401,0.0000,1.0000],[1.0000,0.0000,0.4362,0.5638,0.2675,0.7325,0.0000,1.0000],[1.0000,0.0000,0.5335,0.4665,0.2415,0.7585,0.0000,1.0000],[1.0000,0.0000,0.3406,0.6594,0.2515,0.7485,0.0000,1.0000],[1.0000,0.0000,0.3155,0.6845,0.1845,0.8155,0.0000,1.0000],[1.0000,0.0000,0.3222,0.6778,0.1938,0.8062,0.0000,1.0000],[1.0000,0.0000,0.3174,0.6826,0.1935,0.8065,0.0000,1.0000],[1.0000,0.0000,0.3332,0.6668,0.1941,0.8059,0.0000,1.0000],[1.0000,0.0000,0.5715,0.4285,0.2719,0.7281,0.0000,1.0000],[1.0000,0.0000,0.3296,0.6704,0.1704,0.8296,0.0000,1.0000],[1.0000,0.0000,0.4856,0.5144,0.2711,0.7289,0.0000,1.0000],[1.0000,0.0000,0.5998,0.4002,0.2968,0.7032,0.0000,1.0000],[1.0000,0.0000,0.3401,0.6599,0.1775,0.8225,0.0000,1.0000],[1.0000,0.0000,0.5367,0.4633,0.3472,0.6528,0.0000,1.0000],[1.0000,0.0000,0.5824,0.4176,0.3297,0.6703,0.0000,1.0000],[1.0000,0.0000,0.6330,0.3670,0.3237,0.6763,0.0000,1.0000],[1.0000,0.0000,0.3140,0.6860,0.1860,0.8140,0.0000,1.0000],[0.5000,0.5000,0.0000,1.0000],[1.0000,0.0000,0.3199,0.6801,0.1801,0.8199,0.0000,1.0000],[1.0000,0.0000,0.3535,0.6465,0.2815,0.7185,0.0000,1.0000],[1.0000,0.0000,0.3921,0.6079,0.5074,0.4926,0.2246,0.7754,0.4503,0.5497,0.1675,0.8325,0.2828,0.7172,0.0000,1.0000],[1.0000,0.0000,0.6297,0.3703,0.3539,0.6461,0.0000,1.0000],[1.0000,0.0000,0.4097,0.5903,0.3353,0.6647,0.0000,1.0000],[1.0000,0.0000,0.5833,0.4167,0.5798,0.4202,0.3439,0.6561,0.4753,0.5247,0.2394,0.7606,0.2359,0.7641,0.0000,1.0000],[1.0000,0.0000,0.4972,0.5028,0.5262,0.4738,0.3121,0.6879,0.3992,0.6008,0.1851,0.8149,0.2141,0.7859,0.0000,1.0000],[1.0000,0.0000,0.4178,0.5822,0.2722,0.7278,0.0000,1.0000],[1.0000,0.0000,0.4776,0.5224,0.3144,0.6856,0.0000,1.0000],[1.0000,0.0000,0.3530,0.6470,0.2893,0.7107,0.0000,1.0000],[1.0000,0.0000,0.3441,0.6559,0.2388,0.7612,0.0000,1.0000],[1.0000,0.0000,0.2793,0.7207,0.2207,0.7793,0.0000,1.0000],[1.0000,0.0000,0.3769,0.6231,0.3181,0.6819,0.0000,1.0000],[1.0000,0.0000,0.5096,0.4904,0.3424,0.6576,0.0000,1.0000],[1.0000,0.0000,0.6104,0.3896,0.3434,0.6566,0.0000,1.0000],[1.0000,0.0000,0.3109,0.6891,0.2341,0.7659,0.0000,1.0000],[1.0000,0.0000,0.3852,0.6148,0.2498,0.7502,0.0000,1.0000],[1.0000,0.0000,0.2558,0.7442,0.2442,0.7558,0.0000,1.0000],[1.0000,0.0000,0.2454,0.7546,0.2546,0.7454,0.0000,1.0000],[1.0000,0.0000,0.2324,0.7676,0.2676,0.7324,0.0000,1.0000],[1.0000,0.0000,0.2399,0.7601,0.2601,0.7399,0.0000,1.0000],[1.0000,0.0000,0.2500,0.7500,0.2500,0.7500,0.0000,1.0000],[1.0000,0.0000,0.2588,0.7412,0.2412,0.7588,0.0000,1.0000],[1.0000,0.0000,0.2912,0.7088,0.2588,0.7412,0.0000,1.0000],[1.0000,0.0000,0.3523,0.6477,0.4480,0.5520,0.1983,0.8017,0.4036,0.5964,0.1539,0.8461,0.2497,0.7503,0.0000,1.0000],[1.0000,0.0000,0.4957,0.5043,0.2743,0.7257,0.0000,1.0000],[1.0000,0.0000,0.3224,0.6776,0.3575,0.6425,0.1798,0.8202,0.3202,0.6798,0.1425,0.8575,0.1776,0.8224,0.0000,1.0000],[1.0000,0.0000,0.2838,0.7162,0.2162,0.7838,0.0000,1.0000],[1.0000,0.0000,0.4273,0.5727,0.4441,0.5559,0.2546,0.7454,0.3622,0.6378,0.1727,0.8273,0.1895,0.8105,0.0000,1.0000],[1.0000,0.0000,0.3294,0.6706,0.2729,0.7271,0.0000,1.0000],[1.0000,0.0000,0.2813,0.7187,0.2187,0.7813,0.0000,1.0000],[1.0000,0.0000,0.3223,0.6777,0.3576,0.6424,0.1799,0.8201,0.3201,0.6799,0.1424,0.8576,0.1777,0.8223,0.0000,1.0000],[1.0000,0.0000,0.5173,0.4827,0.2898,0.7102,0.0000,1.0000],[1.0000,0.0000,0.3889,0.6111,0.1842,0.8158,0.0000,1.0000],[1.0000,0.0000,0.6392,0.3608,0.3616,0.6384,0.0000,1.0000],[1.0000,0.0000,0.5543,0.4457,0.3774,0.6226,0.0000,1.0000]]
cpt_4to5 = [[1.0000,0.0000,0.4963,0.5037,0.5668,0.4332,0.3824,0.6176,0.2982,0.7018,0.1139,0.8861,0.1844,0.8156,0.0000,1.0000],[1.0000,0.0000,0.5250,0.4750,0.4859,0.5141,0.3333,0.6667,0.3442,0.6558,0.1916,0.8084,0.1526,0.8474,0.0000,1.0000],[1.0000,0.0000,0.4514,0.5486,0.2153,0.7847,0.0000,1.0000],[1.0000,0.0000,0.4238,0.5762,0.2429,0.7571,0.0000,1.0000],[1.0000,0.0000,0.4154,0.5846,0.2512,0.7488,0.0000,1.0000],[1.0000,0.0000,0.4120,0.5880,0.2763,0.7237,0.0000,1.0000],[1.0000,0.0000,0.4435,0.5565,0.2560,0.7440,0.0000,1.0000],[1.0000,0.0000,0.4883,0.5117,0.2171,0.7829,0.0000,1.0000],[1.0000,0.0000,0.4605,0.5395,0.5364,0.4636,0.3127,0.6873,0.3715,0.6285,0.1478,0.8522,0.2237,0.7763,0.0000,1.0000],[1.0000,0.0000,0.5315,0.4685,0.5086,0.4914,0.3443,0.6557,0.3514,0.6486,0.1872,0.8128,0.1642,0.8358,0.0000,1.0000],[1.0000,0.0000,0.4398,0.5602,0.2269,0.7731,0.0000,1.0000],[1.0000,0.0000,0.4209,0.5791,0.2458,0.7542,0.0000,1.0000],[1.0000,0.0000,0.4155,0.5845,0.2512,0.7488,0.0000,1.0000],[1.0000,0.0000,0.4589,0.5411,0.2077,0.7923,0.0000,1.0000],[1.0000,0.0000,0.3849,0.6151,0.2818,0.7182,0.0000,1.0000],[1.0000,0.0000,0.4207,0.5793,0.2459,0.7541,0.0000,1.0000],[1.0000,0.0000,0.4296,0.5704,0.2370,0.7630,0.0000,1.0000],[1.0000,0.0000,0.4233,0.5767,0.2434,0.7566,0.0000,1.0000],[1.0000,0.0000,0.4314,0.5686,0.2353,0.7647,0.0000,1.0000],[1.0000,0.0000,0.4509,0.5491,0.2158,0.7842,0.0000,1.0000],[1.0000,0.0000,0.4394,0.5606,0.2272,0.7728,0.0000,1.0000],[1.0000,0.0000,0.4288,0.5712,0.2378,0.7622,0.0000,1.0000],[1.0000,0.0000,0.4468,0.5532,0.2199,0.7801,0.0000,1.0000],[1.0000,0.0000,0.4535,0.5465,0.2132,0.7868,0.0000,1.0000],[1.0000,0.0000,0.4066,0.5934,0.2601,0.7399,0.0000,1.0000],[1.0000,0.0000,0.4291,0.5709,0.2376,0.7624,0.0000,1.0000],[1.0000,0.0000,0.4419,0.5581,0.2248,0.7752,0.0000,1.0000],[1.0000,0.0000,0.4187,0.5813,0.2480,0.7520,0.0000,1.0000],[0.6667,0.3333,0.0000,1.0000],[1.0000,0.0000,0.4266,0.5734,0.2401,0.7599,0.0000,1.0000],[1.0000,0.0000,0.3711,0.6289,0.2956,0.7044,0.0000,1.0000],[1.0000,0.0000,0.4202,0.5798,0.4880,0.5120,0.2415,0.7585,0.4251,0.5749,0.1787,0.8213,0.2465,0.7535,0.0000,1.0000],[1.0000,0.0000,0.4284,0.5716,0.2383,0.7617,0.0000,1.0000],[1.0000,0.0000,0.3666,0.6334,0.3001,0.6999,0.0000,1.0000],[1.0000,0.0000,0.4848,0.5152,0.4684,0.5316,0.2866,0.7134,0.3801,0.6199,0.1982,0.8018,0.1818,0.8182,0.0000,1.0000],[1.0000,0.0000,0.4848,0.5152,0.4878,0.5122,0.3060,0.6940,0.3607,0.6393,0.1789,0.8211,0.1818,0.8182,0.0000,1.0000],[1.0000,0.0000,0.4037,0.5963,0.2630,0.7370,0.0000,1.0000],[1.0000,0.0000,0.4030,0.5970,0.2636,0.7364,0.0000,1.0000],[1.0000,0.0000,0.3677,0.6323,0.2990,0.7010,0.0000,1.0000],[1.0000,0.0000,0.3922,0.6078,0.2745,0.7255,0.0000,1.0000],[1.0000,0.0000,0.3724,0.6276,0.2943,0.7057,0.0000,1.0000],[1.0000,0.0000,0.3615,0.6385,0.3051,0.6949,0.0000,1.0000],[1.0000,0.0000,0.3997,0.6003,0.2670,0.7330,0.0000,1.0000],[1.0000,0.0000,0.4306,0.5694,0.2360,0.7640,0.0000,1.0000],[1.0000,0.0000,0.3803,0.6197,0.2864,0.7136,0.0000,1.0000],[1.0000,0.0000,0.4044,0.5956,0.2623,0.7377,0.0000,1.0000],[1.0000,0.0000,0.3410,0.6590,0.3256,0.6744,0.0000,1.0000],[1.0000,0.0000,0.3272,0.6728,0.3395,0.6605,0.0000,1.0000],[1.0000,0.0000,0.3098,0.6902,0.3568,0.6432,0.0000,1.0000],[1.0000,0.0000,0.3198,0.6802,0.3468,0.6532,0.0000,1.0000],[1.0000,0.0000,0.3333,0.6667,0.3333,0.6667,0.0000,1.0000],[1.0000,0.0000,0.3451,0.6549,0.3216,0.6784,0.0000,1.0000],[1.0000,0.0000,0.3529,0.6471,0.3137,0.6863,0.0000,1.0000],[1.0000,0.0000,0.4697,0.5303,0.4614,0.5386,0.2645,0.7355,0.4022,0.5978,0.2052,0.7948,0.1970,0.8030,0.0000,1.0000],[1.0000,0.0000,0.4292,0.5708,0.2375,0.7625,0.0000,1.0000],[1.0000,0.0000,0.4298,0.5702,0.4766,0.5234,0.2398,0.7602,0.4269,0.5731,0.1901,0.8099,0.2368,0.7632,0.0000,1.0000],[1.0000,0.0000,0.3784,0.6216,0.2882,0.7118,0.0000,1.0000],[1.0000,0.0000,0.5247,0.4753,0.4592,0.5408,0.3172,0.6828,0.3494,0.6506,0.2075,0.7925,0.1419,0.8581,0.0000,1.0000],[1.0000,0.0000,0.3660,0.6340,0.3007,0.6993,0.0000,1.0000],[1.0000,0.0000,0.3751,0.6249,0.2915,0.7085,0.0000,1.0000],[1.0000,0.0000,0.4297,0.5703,0.4768,0.5232,0.2399,0.7601,0.4268,0.5732,0.1898,0.8102,0.2369,0.7631,0.0000,1.0000],[1.0000,0.0000,0.4311,0.5689,0.2356,0.7644,0.0000,1.0000],[1.0000,0.0000,0.4549,0.5451,0.2118,0.7882,0.0000,1.0000],[1.0000,0.0000,0.4326,0.5674,0.2410,0.7590,0.0000,1.0000],[1.0000,0.0000,0.4175,0.5825,0.2516,0.7484,0.0000,1.0000]]
cpt_6to7 = [[1.0000,0.0000,0.5426,0.4574,0.6376,0.3624,0.4302,0.5698,0.3198,0.6802,0.1124,0.8876,0.2074,0.7926,0.0000,1.0000],[1.0000,0.0000,0.5784,0.4216,0.5466,0.4534,0.3750,0.6250,0.3750,0.6250,0.2034,0.7966,0.1716,0.8284,0.0000,1.0000],[1.0000,0.0000,0.5078,0.4922,0.2422,0.7578,0.0000,1.0000],[1.0000,0.0000,0.4768,0.5232,0.2732,0.7268,0.0000,1.0000],[1.0000,0.0000,0.4674,0.5326,0.2826,0.7174,0.0000,1.0000],[1.0000,0.0000,0.4635,0.5365,0.2865,0.7135,0.0000,1.0000],[1.0000,0.0000,0.4820,0.5180,0.2680,0.7320,0.0000,1.0000],[1.0000,0.0000,0.5206,0.4794,0.2294,0.7706,0.0000,1.0000],[1.0000,0.0000,0.5181,0.4819,0.5837,0.4163,0.3518,0.6482,0.3982,0.6018,0.1663,0.8337,0.2319,0.7681,0.0000,1.0000],[1.0000,0.0000,0.5979,0.4021,0.5394,0.4606,0.3873,0.6127,0.3627,0.6373,0.2106,0.7894,0.1521,0.8479,0.0000,1.0000],[1.0000,0.0000,0.4948,0.5052,0.2552,0.7448,0.0000,1.0000],[1.0000,0.0000,0.4735,0.5265,0.2765,0.7235,0.0000,1.0000],[1.0000,0.0000,0.4674,0.5326,0.2826,0.7174,0.0000,1.0000],[1.0000,0.0000,0.5163,0.4837,0.2337,0.7663,0.0000,1.0000],[1.0000,0.0000,0.4330,0.5670,0.3170,0.6830,0.0000,1.0000],[1.0000,0.0000,0.4733,0.5267,0.2767,0.7233,0.0000,1.0000],[1.0000,0.0000,0.4833,0.5167,0.2667,0.7333,0.0000,1.0000],[1.0000,0.0000,0.4762,0.5238,0.2738,0.7262,0.0000,1.0000],[1.0000,0.0000,0.4853,0.5147,0.2647,0.7353,0.0000,1.0000],[1.0000,0.0000,0.5072,0.4928,0.2428,0.7572,0.0000,1.0000],[1.0000,0.0000,0.4944,0.5056,0.2556,0.7444,0.0000,1.0000],[1.0000,0.0000,0.4824,0.5176,0.2676,0.7324,0.0000,1.0000],[1.0000,0.0000,0.5027,0.4973,0.2473,0.7527,0.0000,1.0000],[1.0000,0.0000,0.5102,0.4898,0.2398,0.7602,0.0000,1.0000],[1.0000,0.0000,0.4574,0.5426,0.2926,0.7074,0.0000,1.0000],[1.0000,0.0000,0.4827,0.5173,0.2673,0.7327,0.0000,1.0000],[1.0000,0.0000,0.4971,0.5029,0.2529,0.7471,0.0000,1.0000],[1.0000,0.0000,0.4710,0.5290,0.2790,0.7210,0.0000,1.0000],[0.7500,0.2500,0.0000,1.0000],[1.0000,0.0000,0.4799,0.5201,0.2701,0.7299,0.0000,1.0000],[1.0000,0.0000,0.4175,0.5825,0.3325,0.6675,0.0000,1.0000],[1.0000,0.0000,0.4727,0.5273,0.5490,0.4510,0.2717,0.7283,0.4783,0.5217,0.2010,0.7990,0.2773,0.7227,0.0000,1.0000],[1.0000,0.0000,0.4819,0.5181,0.2681,0.7319,0.0000,1.0000],[1.0000,0.0000,0.4124,0.5876,0.3376,0.6624,0.0000,1.0000],[1.0000,0.0000,0.5455,0.4545,0.5270,0.4730,0.3224,0.6776,0.4276,0.5724,0.2230,0.7770,0.2045,0.7955,0.0000,1.0000],[1.0000,0.0000,0.5455,0.4545,0.5488,0.4512,0.3442,0.6558,0.4058,0.5942,0.2012,0.7988,0.2045,0.7955,0.0000,1.0000],[1.0000,0.0000,0.4542,0.5458,0.2958,0.7042,0.0000,1.0000],[1.0000,0.0000,0.4534,0.5466,0.2966,0.7034,0.0000,1.0000],[1.0000,0.0000,0.4137,0.5863,0.3363,0.6637,0.0000,1.0000],[1.0000,0.0000,0.4412,0.5588,0.3088,0.6912,0.0000,1.0000],[1.0000,0.0000,0.4190,0.5810,0.3310,0.6690,0.0000,1.0000],[1.0000,0.0000,0.4067,0.5933,0.3433,0.6567,0.0000,1.0000],[1.0000,0.0000,0.4496,0.5504,0.3004,0.6996,0.0000,1.0000],[1.0000,0.0000,0.4845,0.5155,0.2655,0.7345,0.0000,1.0000],[1.0000,0.0000,0.4278,0.5722,0.3222,0.6778,0.0000,1.0000],[1.0000,0.0000,0.4550,0.5450,0.2950,0.7050,0.0000,1.0000],[1.0000,0.0000,0.3837,0.6163,0.3663,0.6337,0.0000,1.0000],[1.0000,0.0000,0.3681,0.6319,0.3819,0.6181,0.0000,1.0000],[1.0000,0.0000,0.3486,0.6514,0.4014,0.5986,0.0000,1.0000],[1.0000,0.0000,0.3598,0.6402,0.3902,0.6098,0.0000,1.0000],[1.0000,0.0000,0.3750,0.6250,0.3750,0.6250,0.0000,1.0000],[1.0000,0.0000,0.3882,0.6118,0.3618,0.6382,0.0000,1.0000],[1.0000,0.0000,0.3971,0.6029,0.3529,0.6471,0.0000,1.0000],[1.0000,0.0000,0.5284,0.4716,0.5191,0.4809,0.2975,0.7025,0.4525,0.5475,0.2309,0.7691,0.2216,0.7784,0.0000,1.0000],[1.0000,0.0000,0.4828,0.5172,0.2672,0.7328,0.0000,1.0000],[1.0000,0.0000,0.4836,0.5164,0.5362,0.4638,0.2697,0.7303,0.4803,0.5197,0.2138,0.7862,0.2664,0.7336,0.0000,1.0000],[1.0000,0.0000,0.4257,0.5743,0.3243,0.6757,0.0000,1.0000],[1.0000,0.0000,0.5903,0.4097,0.5166,0.4834,0.3569,0.6431,0.3931,0.6069,0.2334,0.7666,0.1597,0.8403,0.0000,1.0000],[1.0000,0.0000,0.4117,0.5883,0.3383,0.6617,0.0000,1.0000],[1.0000,0.0000,0.4220,0.5780,0.3280,0.6720,0.0000,1.0000],[1.0000,0.0000,0.4834,0.5166,0.5364,0.4636,0.2699,0.7301,0.4801,0.5199,0.2136,0.7864,0.2666,0.7334,0.0000,1.0000],[1.0000,0.0000,0.4850,0.5150,0.2650,0.7350,0.0000,1.0000],[1.0000,0.0000,0.5118,0.4882,0.2382,0.7618,0.0000,1.0000],[1.0000,0.0000,0.4867,0.5133,0.2633,0.7367,0.0000,1.0000],[1.0000,0.0000,0.4697,0.5303,0.2803,0.7197,0.0000,1.0000]]


In [ ]:
for i in range(len(ts_0)):
    ftts_dbn.cpt(ts_0[i]).fillWith(cpt_0[i])

In [ ]:
for n in range(65):
    for i in range(1*n, 1*(n+1)):
        ftts_dbn.cpt(ts_1[i]).fillWith(cpt_1[n])
    for j in range(2*n, 2*(n+1)):
        ftts_dbn.cpt(ts_2to3[j]).fillWith(cpt_2to3[n])
        ftts_dbn.cpt(ts_4to5[j]).fillWith(cpt_4to5[n])
        ftts_dbn.cpt(ts_6to7[j]).fillWith(cpt_6to7[n])

In [ ]:
il = gum.LazyPropagation(ftts_dbn)
iv = gum.VariableElimination(ftts_dbn)

In [ ]:
iv.setEvidence({'r36_0':0, 'r36_1':0, 'r36_2':0, 'r36_3':0, 'r36_4':0, 'r36_5':0, 'r36_6':0, 'r36_7':0 })

In [ ]:
gnb.flow.clear()
for i in [1,3,5,7]:
    gnb.flow.add_html(gnb.getPosterior(ftts_dbn, target = "r65_{}".format(i), evs={'r36_0':0, 'r36_1':0, 'r36_2':0, 'r36_3':0, 'r36_4':0, 'r36_5':0, 'r36_6':0, 'r36_7':0 }), "$P(a{})$".format(i))
gnb.flow.display()

In [ ]:
ftts_dbn_rooms =[e.split('_')[0]+('_') for e in ftts_dbn_nodes]

plt.rcParams['figure.figsize'] = (10, 2)
gdyn.plotFollowUnrolled(ftts_dbn_rooms,ftts_dbn,T=8,evs={'r36_0':0, 'r36_1':0, 'r36_2':0, 'r36_3':0, 'r36_4':0, 'r36_5':0, 'r36_6':0, 'r36_7':0 })

for node in dBN_nodes:
    dag_dBN.cpt(node)
    print(dag_dBN.cpt(node))

dag_dBN.cpt("r1_0").fillWith([0.74, 0.26, 0, 1])


dag_dBN.cpt("r53_0").names # 부모노드를 출력한다. 조건부확률의 조건


dag_dBN.cpt("r38_0").names


gnb.flow.clear()
for i in range(T):
    gnb.flow.add_html(gnb.getPosterior(unroll_dBN, target = "r1_{}".format(i), evs={}), "$P(a{})$".format(i))
gnb.flow.display()

sorted(ftts_dbn.names())

In [ ]:
ftts_dbn.cpt('r60_6')